In [31]:
# %% library

import torch                               # PyTorch kütüphanesi, tensor işlemleri
import torch.nn as nn                      # Yapay sinir ağı katmanlarını tanımlamak için
import torch.optim as optim                # Optimizasyon algoritmalarını içeren modül
import torchvision                         # Görüntü işleme ve pre-defined modelleri içerir
import torchvision.transforms as transforms  # Görüntü dönüşümleri yapmak için
import matplotlib.pyplot as plt            # Görselleştirme

# optional: cihazı belirle (GPU vs CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [32]:
def get_data_loaders(batch_size=64):
    # Görüntüleri tensöre çevir ve normalize et:
    # 1. ToTensor(): Görüntüyü [0, 255] aralığından [0.0, 1.0] aralığına çevirir ve (H x W x C) → (C x H x W) yapar
    # 2. Normalize(mean=[0.5], std=[0.5]): Her pikseli (x - 0.5) / 0.5 ile normalize eder → [0,1] → [-1,1] aralığına çeker
    transform = transforms.Compose([
        transforms.ToTensor(),  
        transforms.Normalize(mean=[0.5], std=[0.5])#compose belirli dönüşüm zinciir oluşturmak için kullanılır
    ])

    train_set = torchvision.datasets.MNIST(root="./data", train=True,download=True ,transform=transform)
    test_set = torchvision.datasets.MNIST(root="./data", train= False,download=True ,transform=transform)
    
    train_loader = torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True)
    test_loader = torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True)
    return train_loader,test_loader
train_loader,test_loader= get_data_loaders()

data visualization

In [33]:
import matplotlib
matplotlib.use("TkAgg")      # GUI gerekmez, çizimler hücrede gösterilir
import matplotlib.pyplot as plt
      # ③ pyplot artık doğru backend ile


# data visualization
def visualize_samples(loader, n):
    # İlk batch'ten görüntüleri ve etiketleri alalım
    images, labels = next(iter(loader))

    # n adet görseli yatay şekilde göstermek için subplot oluştur
    fig, axes = plt.subplots(1, n, figsize=(10, 5))  # n farklı görüntü için görselleştirme alanı

    for i in range(n):
        # Görseli sıkıştır (tek kanallı hale getir), gri tonlamalı olarak göster
        axes[i].imshow(images[i].squeeze(), cmap="gray")
        
        # Başlığa etiket yaz
        axes[i].set_title(f"Label: {labels[i].item()}")
        
        # Eksenleri gizle
        axes[i].axis("off")
    
    plt.show()
visualize_samples(test_loader , 5)


yapay sinir ağı

In [34]:
# 'NeuralNetwork' adında bir sınıf tanımlıyoruz. Bu sınıf, PyTorch'un temel sinir ağı modülü olan 'nn.Module' sınıfından miras alır.
# Bu, PyTorch'un bu sınıfı bir sinir ağı modeli olarak tanımasını sağlar.
class NeuralNetwork(nn.Module):
    
    # Bu, sınıfın yapıcı (constructor) metodudur. Sinir ağının katmanları burada tanımlanır.
    def __init__(self):
        # 'super()' fonksiyonu, 'nn.Module' ana sınıfının yapıcı metodunu çağırır. Bu, modelin doğru bir şekilde kurulması için gereklidir.
        super(NeuralNetwork, self).__init__()
        
        # 'nn.Flatten()' katmanını oluşturuyoruz. Bu katman, çok boyutlu girdileri (örneğin 28x28 piksellik bir resim) tek boyutlu bir vektöre dönüştürür.
        self.flatten = nn.Flatten()
        
        # İlk tam bağlantılı (fully connected) katmanı ('Linear' katman) tanımlıyoruz.
        # Bu katman 28*28 = 784 adet girdiyi alır ve bunları 128 adet çıktıya dönüştürür.
        # Genellikle MNIST gibi 28x28 piksellik resim veri setleri için kullanılır.
        self.fc1 = nn.Linear(28*28, 128)
        
        # ReLU (Rectified Linear Unit) aktivasyon fonksiyonunu tanımlıyoruz.
        # Bu fonksiyon, modele non-linear (doğrusal olmayan) bir özellik kazandırır. Negatif değerleri sıfır yapar, pozitif değerleri ise olduğu gibi bırakır.
        self.relu = nn.ReLU()
        
        # İkinci tam bağlantılı katmanı tanımlıyoruz.
        # Bir önceki katmanın çıktısı olan 128 özelliği alır ve bunları 64 özelliğe indirger.
        self.fc2 = nn.Linear(128, 64)
        
        # !!! DİKKAT: HATA POTANSİYELİ !!!
        # Üçüncü tam bağlantılı katmanı tanımlıyoruz. Bu katman 54 girdi alacak şekilde tanımlanmış.
        # Ancak bir önceki katman olan 'self.fc2', 64 adet çıktı üretir.
        # Girdi (54) ve çıktı (64) boyutları uyuşmadığı için bu kod 'forward' metodunda hata verecektir.
        # Doğrusu: self.fc3 = nn.Linear(64, 10) olmalıydı.
        self.fc3 = nn.Linear(64, 10)
        

    # 'forward' metodu, verinin (x) ağ içinde nasıl akacağını tanımlar (ileri yayılım - forward propagation).
    def forward(self, x):
        # Girdi verisi 'x'i 'flatten' katmanından geçirerek düzleştiriyoruz.
        # Örneğin [batch_size, 1, 28, 28] boyutundaki bir resim tensörü [batch_size, 784] boyutunda bir vektöre dönüşür.
        x = self.flatten(x)
        
        # Düzleştirilmiş veriyi ilk tam bağlantılı katman olan 'fc1'den geçiriyoruz.
        x = self.fc1(x) 
        
        # 'fc1' katmanının çıktısını 'relu' aktivasyon fonksiyonundan geçiriyoruz.
        x = self.relu(x)
        
        # Aktivasyon fonksiyonundan geçen veriyi ikinci tam bağlantılı katman olan 'fc2'den geçiriyoruz.
        # Bu noktada 'x'in boyutu [batch_size, 64] olur.
        x = self.fc2(x)
        
        # 'fc2'den çıkan 64 boyutlu tensör, 54 girdi bekleyen 'fc3'e gönderilmeye çalışılacağı için burada bir boyut uyuşmazlığı hatası ('runtime error') alınacaktır.
        x = self.fc3(x)
        
        # Bu satır gereksizdir ve muhtemelen bir hatadır.
        # 'fc3' katmanının çıktısı zaten [batch_size, 10] boyutunda düz bir tensördür.
        # Zaten düz olan bir veriyi tekrar düzleştirmeye çalışmak anlamsızdır ve genellikle bir etkisi olmaz.
        x = self.flatten(x)
        
        # Metodun sonunda, ağın son çıktısı olan 'x' döndürülür (hata olmasaydı).
        return x
# Gerekli kütüphaneleri içe aktardığımızı varsayalım.
# import torch.nn as nn
# import torch.optim as optim

# 'define_loss_and_optimizer' adında bir lambda (anonim) fonksiyon tanımlıyoruz.
# Bu fonksiyon, parametre olarak bir 'model' (sinir ağı modeli) alır.
define_loss_and_optimizer = lambda model: (
    
    # Fonksiyonun döndüreceği ilk eleman: Kayıp Fonksiyonu (Loss Function).
    # 'nn.CrossEntropyLoss()', çok sınıflı sınıflandırma problemleri için yaygın olarak kullanılan bir kayıp fonksiyonudur.
    # Bu fonksiyon, kendi içinde bir 'LogSoftmax' ve 'NLLLoss' (Negative Log Likelihood Loss) katmanlarını birleştirir.
    # Bu nedenle, modelin son katmanında ayrıca bir Softmax fonksiyonu kullanmaya gerek yoktur.
    # Modelin tahminlerinin (logits) gerçek etiketlerden ne kadar uzak olduğunu ölçer.
    nn.CrossEntropyLoss(),  # multi class classification problems loss function

    # Fonksiyonun döndüreceği ikinci eleman: Optimizasyon Algoritması (Optimizer).
    # 'optim.Adam', Stokastik Gradyan İnişi'nin (SGD) bir uzantısı olan ve uyarlanabilir öğrenme oranları kullanan popüler bir optimizasyon algoritmasıdır.
    # Parametreler:
    # 1. model.parameters(): Optimize edilecek olan parametreleri (ağın öğrenilebilir ağırlıkları ve bias'ları) belirtir.
    # 2. lr = 0.001: Öğrenme oranını (learning rate) belirtir. Bu, her adımda modelin ağırlıklarının ne kadar güncelleneceğini kontrol eden bir hiperparametredir.
    optim.Adam(model.parameters(), lr = 0.001)
)
model = NeuralNetwork().to(device)

traiin model


In [35]:
# Gerekli kütüphaneleri içe aktardığımızı varsayalım.
# import torch
# import matplotlib.pyplot as plt

# %% train başlığı sadece bir hücre ayıracıdır, genellikle Jupyter Notebook veya benzeri ortamlarda kullanılır.

# Modelin eğitimini gerçekleştirecek olan fonksiyonu tanımlıyoruz.
# Parametreler:
# model: Eğitilecek PyTorch modeli.
# train_loader: Eğitim verilerini içeren DataLoader nesnesi. Verileri mini-batch'ler halinde sunar.
# criterion: Kayıp fonksiyonu (örneğin nn.CrossEntropyLoss).
# optimizer: Optimizasyon algoritması (örneğin optim.Adam).
# epochs: Veri setinin tamamı üzerinden kaç kez geçileceği (eğitim turu sayısı). Varsayılan değeri 10'dur.
def train_model(model, train_loader, criterion, optimizer, epochs = 10):

    # model.train() komutu, modeli "eğitim moduna" alır.
    # Bu mod, Dropout ve BatchNorm gibi katmanların eğitim sırasında aktif olmasını sağlar.
    # (Değerlendirme için model.eval() kullanılır.)
    model.train() 
    
    # Her bir epoch (eğitim turu) sonunda hesaplanan ortalama kayıp değerlerini saklamak için boş bir liste oluşturuyoruz.
    train_losses = [] 
    
    # Belirtilen epoch sayısı kadar döngü başlatıyoruz. Bu, ana eğitim döngüsüdür.
    for epoch in range(epochs): 
        # Her epoch'un başında o epoch için toplam kaybı sıfırlıyoruz.
        total_loss = 0 
        
        # 'train_loader' içindeki tüm veri yığınları (batch'ler) üzerinde iterasyon yapıyoruz.
        # Her adımda, loader bize bir yığın resim ('images') ve bunlara karşılık gelen etiketleri ('labels') verir.
        for images, labels in train_loader: 
            
            # Veri ve etiketleri, hesaplamanın yapılacağı cihaza (CPU veya GPU) taşıyoruz.
            # 'device' değişkeninin daha önce tanımlanmış olması gerekir (örn: device = "cuda" if torch.cuda.is_available() else "cpu").
            images, labels = images.to(device), labels.to(device) 
            
            # Gradyanları sıfırlıyoruz. PyTorch gradyanları her geri yayılımda biriktirir.
            # Bu yüzden her yeni yığın için hesaplama yapmadan önce eski gradyanları temizlememiz gerekir.
            optimizer.zero_grad() 
            
            # Modeli kullanarak girdilerden (images) tahminler (predictions) üretiyoruz.
            # Bu işlem, verinin ağ içinden ileri doğru akışını (forward propagation) temsil eder.
            predictions = model(images) 
            
            # Kayıp fonksiyonunu (criterion) kullanarak modelin tahminleri (predictions) ile gerçek etiketler (labels) arasındaki hatayı (kaybı) hesaplıyoruz.
            loss = criterion(predictions, labels) 
            
            # 'loss.backward()' metodu, hesaplanan kayba göre modelin tüm öğrenilebilir parametreleri için gradyanları hesaplar (geri yayılım - backpropagation).
            loss.backward() 
            
            # 'optimizer.step()' metodu, hesaplanan gradyanları kullanarak modelin ağırlıklarını günceller.
            optimizer.step() 
            
            # O anki yığının kaybını (.item() ile tensörden Python sayısına çevirerek) toplam kayba ekliyoruz.
            total_loss = total_loss + loss.item() 
        
        # Epoch'taki tüm yığınlar işlendikten sonra, ortalama kaybı hesaplıyoruz.
        # Toplam kayıp, veri yükleyicideki yığın sayısına bölünür.
        avg_loss = total_loss / len(train_loader) 
        
        # Hesaplanan ortalama kaybı, daha sonra grafiğini çizmek üzere listemize ekliyoruz.
        train_losses.append(avg_loss)
        
        # Mevcut epoch numarasını ve o epoch'a ait ortalama kaybı ekrana yazdırıyoruz.
        # f-string formatı, {avg_loss:.3f} ile kaybı ondalık noktadan sonra 3 basamakla gösterir.
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.3f}")

    # Eğitim döngüsü bittikten sonra kayıp grafiğini çizmek için bölüm.
    # Yeni bir çizim figürü oluştur.
    plt.figure() 
    
    # Kayıp grafiğini çiz.
    # x ekseni: epoch sayıları (1'den başlar), y ekseni: her epoch'taki kayıp değerleri.
    # marker='o': her veri noktasını daire ile işaretle.
    # linestyle='-': noktaları düz bir çizgiyle birleştir.
    # label='Train Loss': grafiğin etiketini belirle.
    plt.plot(range(1, epochs + 1), train_losses, marker = "o", linestyle = "-", label = "Train Loss")
    
    # x ekseninin etiketini ayarla.
    plt.xlabel("Epochs") 
    
    # y ekseninin etiketini ayarla.
    plt.ylabel("Loss") 
    
    # Grafiğin başlığını ayarla.
    plt.title("Training Loss")
    
    # Grafikteki etiketleri ('Train Loss') göstermek için bir lejant ekle.
    plt.legend()
    
    # Hazırlanan grafiği ekranda göster.
    plt.show()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [36]:
# %% test başlığı, genellikle bir sonraki kod hücresinin "test" ile ilgili olduğunu belirtmek için kullanılır.

# Daha önce tanımladığımız 'train_model' fonksiyonunu burada çağırıyoruz.
# Bu satır, asıl eğitim sürecini başlatan komuttur.
# Fonksiyona aşağıdaki argümanları iletiyoruz:
# 1. model: Eğitmek istediğimiz, daha önceden oluşturulmuş sinir ağı modeli.
# 2. train_loader: Eğitim verilerini yığınlar (batch) halinde içeren DataLoader nesnesi.
# 3. criterion: Modelin hatasını ölçmek için tanımlanmış kayıp fonksiyonu (örneğin CrossEntropyLoss).
# 4. optimizer: Modelin ağırlıklarını güncellemek için tanımlanmış optimizasyon algoritması (örneğin Adam).
# 5. epochs=5: Eğitim sürecinin toplam 5 tur (epoch) süreceğini belirtiyoruz. 
#    Veri setinin tamamı üzerinden 5 kez geçilecektir.
train_model(model, train_loader, criterion, optimizer, epochs=5)

Epoch 1/5, Loss: 0.368
Epoch 2/5, Loss: 0.170
Epoch 3/5, Loss: 0.129
Epoch 4/5, Loss: 0.109
Epoch 5/5, Loss: 0.093


In [37]:
# Gerekli kütüphaneleri içe aktardığımızı varsayalım.
# import torch

# %% test başlığı, bu hücrenin test işlemleriyle ilgili olduğunu belirtir.

# Modelin test performansını ölçecek olan fonksiyonu tanımlıyoruz.
# Parametreler:
# model: Değerlendirilecek, eğitilmiş PyTorch modeli.
# test_loader: Test verilerini içeren DataLoader nesnesi.
def test_model(model, test_loader):

    # model.eval() komutu, modeli "değerlendirme (evaluation) moduna" alır.
    # Bu mod, eğitimde kullanılan Dropout gibi katmanları devre dışı bırakır.
    # Bu, test sırasında tutarlı sonuçlar almak için çok önemlidir.
    model.eval()

    # Doğru tahmin edilen veri sayısını tutmak için bir sayaç başlatıyoruz.
    correct = 0
    # Toplam veri sayısını tutmak için bir sayaç başlatıyoruz.
    total = 0

    # 'with torch.no_grad():' bloğu, bu blok içindeki tüm işlemler için gradyan hesaplamasını devre dışı bırakır.
    # Test/değerlendirme sırasında ağırlıkları güncellemeyeceğimiz için gradyanlara ihtiyacımız yoktur.
    # Bu, gereksiz hesaplamaları önleyerek süreci hızlandırır ve bellek kullanımını azaltır.
    with torch.no_grad():
        # 'test_loader' içindeki tüm veri yığınları (batch'ler) üzerinde iterasyon yapıyoruz.
        for images, labels in test_loader:
            # Test verilerini ve etiketlerini, hesaplamanın yapılacağı cihaza (CPU veya GPU) taşıyoruz.
            images, labels = images.to(device), labels.to(device)

            # Modeli kullanarak girdilerden (images) tahminler (predictions) üretiyoruz (ileri yayılım).
            predictions = model(images)

            # Modelin çıktısı olan olasılık/skorlardan en yüksek olanın indeksini alıyoruz.
            # torch.max(predictions, 1) fonksiyonu, her bir örnek için en yüksek değere sahip sınıfın skorunu ve indeksini döndürür.
            # 'predictions' tensörünün 1. boyutu (sınıf skorları) üzerinde çalışır.
            # Bize sadece indeks (tahmin edilen sınıf) gerektiği için skorları '_' ile görmezden geliyoruz.
            _, predicted = torch.max(predictions, 1)

            # Toplam veri sayacını, mevcut yığındaki (batch) etiket sayısı kadar artırıyoruz.
            # labels.size(0) o anki yığının boyutunu (batch size) verir.
            total += labels.size(0)

            # Doğru tahmin sayacını güncelliyoruz.
            # (predicted == labels) işlemi, tahminler ile gerçek etiketleri karşılaştırarak bir boolean tensör ([True, False, True...]) üretir.
            # .sum() bu tensördeki 'True' (1) değerlerini toplayarak o yığındaki doğru tahmin sayısını bulur.
            # .item() ise bu sonucu tek bir sayı olarak Python'a aktarır.
            correct += (predicted == labels).sum().item()

    # Tüm test verileri işlendikten sonra, modelin doğruluk oranını (accuracy) hesaplayıp yazdırıyoruz.
    # Doğruluk = (Doğru Tahmin Sayısı / Toplam Veri Sayısı) * 100
    # {değer:.3f} formatı, sonucu ondalık noktadan sonra 3 basamakla gösterir.
    print(f"Test Accuracy: {100 * correct / total:.3f}%")

# Eğitilmiş 'model' ve 'test_loader' ile test fonksiyonunu çağırıyoruz.
# Bu satırın çalışması için 'model'in eğitilmiş ve 'test_loader'ın tanımlanmış olması gerekir.
test_model(model, test_loader)

Test Accuracy: 97.178%


In [38]:
# if __name__ == "__main__": bloğu, Python'da standart bir yapıdır.
# Bu blok içindeki kodlar, sadece bu script dosyası doğrudan çalıştırıldığında yürütülür.
# Eğer bu script başka bir dosya tarafından 'import' edilirse, bu blok içindeki kodlar çalışmaz.
# Bu, kodun modüler olmasını ve ana çalışma mantığının sadece istendiğinde tetiklenmesini sağlar.
if __name__ == "__main__":

    # 'get_data_loaders()' adında bir fonksiyonu çağırıyoruz (bu fonksiyonun daha önce tanımlandığını varsayıyoruz).
    # Bu fonksiyonun görevi, eğitim ve test veri setlerini yükleyip hazırlamak ve bunları
    # PyTorch'un DataLoader nesnelerine dönüştürmektir.
    # Sonuç olarak, eğitim için 'train_loader' ve test için 'test_loader' değişkenlerini elde ederiz.
    train_loader, test_loader = get_data_loaders()

    # 'visualize_samples()' adında bir fonksiyonu çağırıyoruz (bu da önceden tanımlanmış olmalı).
    # Bu fonksiyon, muhtemelen 'train_loader'dan 5 adet veri örneği alıp bunları ekranda görselleştirir.
    # Bu, verinin doğru yüklenip yüklenmediğini kontrol etmek için yapılan yaygın bir işlemdir.
    visualize_samples(train_loader, 5)

    # Daha önce tanımladığımız 'NeuralNetwork' sınıfından bir nesne (yani modelimizi) oluşturuyoruz.
    # .to(device) metodu ile modeli, hesaplamaların yapılacağı cihaza (CPU veya GPU) taşıyoruz.
    model = NeuralNetwork().to(device)

    # Kayıp fonksiyonunu (criterion) ve optimizasyon algoritmasını (optimizer) tanımlamak için
    # daha önce oluşturduğumuz 'define_loss_and_optimizer' fonksiyonunu çağırıyoruz.
    criterion, optimizer = define_loss_and_optimizer(model)

    # Modelin eğitim sürecini başlatmak için 'train_model' fonksiyonunu çağırıyoruz.
    # Bu fonksiyona gerekli olan tüm bileşenleri (model, veri, kayıp fonksiyonu, optimizer) veriyoruz.
    # Epoch sayısı belirtilmediği için fonksiyonun varsayılan değeri (örn: 10) kullanılır.
    train_model(model, train_loader, criterion, optimizer)

    # 'train_model' fonksiyonu çalışıp modelin eğitimi bittikten sonra bu satıra geçilir.
    # 'test_model' fonksiyonunu çağırarak, eğitilmiş modelin performansını daha önce hiç görmediği
    # test verileri üzerinde ölçüyoruz. Sonuç olarak test doğruluğu ekrana yazdırılır.
    test_model(model, test_loader)

Epoch 1/10, Loss: 0.356
Epoch 2/10, Loss: 0.174
Epoch 3/10, Loss: 0.131
Epoch 4/10, Loss: 0.109
Epoch 5/10, Loss: 0.096
Epoch 6/10, Loss: 0.083
Epoch 7/10, Loss: 0.077
Epoch 8/10, Loss: 0.071
Epoch 9/10, Loss: 0.064
Epoch 10/10, Loss: 0.061
Test Accuracy: 98.235%
